This is the second assignment for the Coursera course "Advanced Machine Learning and Signal Processing"

Again, please insert to code to your ApacheCouchDB based Cloudant instance below using the "Insert Code" function of Watson Studio( you've done this in Assignment 1 before)

Done, just execute all cells one after the other and you are done - just note that in the last one you have to update your email address (the one you've used for coursera) and obtain a submission token, you get this from the programming assignment directly on coursera.

Please fill in the sections labelled with "###YOUR_CODE_GOES_HERE###"

In [19]:
# The code was removed by Watson Studio for sharing.

Let's create a SparkSession object and put the Cloudant credentials into it

In [2]:
spark = SparkSession\
    .builder\
    .appName("Cloudant Spark SQL Example in Python using temp tables")\
    .config("cloudant.host",credentials_1['custom_url'].split('@')[1])\
    .config("cloudant.username", credentials_1['username'])\
    .config("cloudant.password",credentials_1['password'])\
    .config("jsonstore.rdd.partitions", 1)\
    .getOrCreate()

Now it’s time to have a look at the recorded sensor data. You should see data similar to the one exemplified below….


In [3]:
df=spark.read.load('shake_classification', "com.cloudant.spark")

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()


+-----+-------------+-----+-----+-----+--------------------+--------------------+
|CLASS|     SENSORID|    X|    Y|    Z|                 _id|                _rev|
+-----+-------------+-----+-----+-----+--------------------+--------------------+
|    0|Timbake123456|-0.03|-0.03|-0.03|0cd2854f6ed068fa2...|1-82daef796aa82bb...|
|    0|Timbake123456| 0.07| 0.07| 0.07|0cd2854f6ed068fa2...|1-af088a4bae09e45...|
|    0|Timbake123456|-0.01|-0.01|-0.01|0cd2854f6ed068fa2...|1-6c2664ac354e25f...|
|    0|Timbake123456| -0.8| -0.8| -0.8|0cd2854f6ed068fa2...|1-3995781db33db3c...|
|    0|Timbake123456|  0.6|  0.6|  0.6|0cd2854f6ed068fa2...|1-8a9000db7949ed8...|
|    0|Timbake123456|-0.68|-0.68|-0.68|0cd2854f6ed068fa2...|1-20caeaa69d188c9...|
|    0|Timbake123456|-0.15|-0.15|-0.15|0cd2854f6ed068fa2...|1-f573ff812f3d71a...|
|    0|Timbake123456|-0.02|-0.02|-0.02|0cd2854f6ed068fa2...|1-567df98d896c672...|
|    0|Timbake123456|-0.03|-0.03|-0.03|0cd2854f6ed068fa2...|1-82daef796aa82bb...|
|    0|Timbake12

Please create a VectorAssembler which consumed columns X, Y and Z and produces a column “features”


In [5]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols=["X", "Y", "Z"], outputCol="features")

Please insatiate a classifier from the SparkML package and assign it to the classifier variable. Make sure to either
1.	Rename the “CLASS” column to “label” or
2.	Specify the label-column correctly to be “CLASS”


In [9]:
from pyspark.ml.classification import GBTClassifier

classifier = GBTClassifier(labelCol="CLASS", featuresCol="features", maxIter=10)

Let’s train and evaluate…


In [10]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, classifier])

In [11]:
model = pipeline.fit(df)

In [12]:
prediction = model.transform(df)

In [13]:
prediction.show()

+-----+-------------+-----+-----+-----+--------------------+--------------------+-------------------+----------+
|CLASS|     SENSORID|    X|    Y|    Z|                 _id|                _rev|           features|prediction|
+-----+-------------+-----+-----+-----+--------------------+--------------------+-------------------+----------+
|    0|Timbake123456|-0.03|-0.03|-0.03|0cd2854f6ed068fa2...|1-82daef796aa82bb...|[-0.03,-0.03,-0.03]|       0.0|
|    0|Timbake123456| 0.07| 0.07| 0.07|0cd2854f6ed068fa2...|1-af088a4bae09e45...|   [0.07,0.07,0.07]|       0.0|
|    0|Timbake123456|-0.01|-0.01|-0.01|0cd2854f6ed068fa2...|1-6c2664ac354e25f...|[-0.01,-0.01,-0.01]|       0.0|
|    0|Timbake123456| -0.8| -0.8| -0.8|0cd2854f6ed068fa2...|1-3995781db33db3c...|   [-0.8,-0.8,-0.8]|       0.0|
|    0|Timbake123456|  0.6|  0.6|  0.6|0cd2854f6ed068fa2...|1-8a9000db7949ed8...|      [0.6,0.6,0.6]|       0.0|
|    0|Timbake123456|-0.68|-0.68|-0.68|0cd2854f6ed068fa2...|1-20caeaa69d188c9...|[-0.68,-0.68,-0

In [15]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("CLASS")
    
binEval.evaluate(prediction) 

1.0

If you are happy with the result (I’m happy with > 0.55) please submit your solution to the grader by executing the following cells, please don’t forget to obtain an assignment submission token (secret) from the Courera’s graders web page and paste it to the “secret” variable below, including your email address you’ve used for Coursera. (0.55 means that you are performing better than random guesses)


In [16]:
!rm -Rf a2_m2.json

In [17]:
prediction = prediction.repartition(1)
prediction.write.json('a2_m2.json')

In [18]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py

--2018-11-03 19:25:38--  https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2289 (2.2K) [text/plain]
Saving to: ‘rklib.py’

100%[======================================>] 2,289       --.-K/s   in 0s      

2018-11-03 19:25:38 (13.9 MB/s) - ‘rklib.py’ saved [2289/2289]



In [20]:
!zip -r a2_m2.json.zip a2_m2.json

  adding: a2_m2.json/ (stored 0%)
  adding: a2_m2.json/part-00000-69f5fb58-2237-437a-9df7-c9db5a8b06f4.json (deflated 95%)
  adding: a2_m2.json/_SUCCESS (stored 0%)
  adding: a2_m2.json/.part-00000-69f5fb58-2237-437a-9df7-c9db5a8b06f4.json.crc (stored 0%)
  adding: a2_m2.json/._SUCCESS.crc (stored 0%)


In [21]:
!base64 a2_m2.json.zip > a2_m2.json.zip.base64

In [1]:
from rklib import submit
key = "J3sDL2J8EeiaXhILFWw2-g"
part = "G4P6f"
email = "timbake2007@gmail.com"
secret = ###YOUR_CODE_GOES_HERE###"

with open('a2_m2.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, secret, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
